# 🚀 Machine Learning Avancé - Comparaison d'Algorithmes

**Auteur:** Boubacar DABO  
**Date:** Juillet 2025  
**Objectif:** Démonstration comparative de 15+ algorithmes ML sur différents datasets

---

## 📊 Vue d'ensemble

Ce notebook présente une analyse comparative complète des algorithmes de Machine Learning les plus utilisés en industrie, incluant:

- **Algorithmes classiques:** Random Forest, SVM, Logistic Regression
- **Gradient Boosting:** XGBoost, LightGBM, CatBoost
- **Neural Networks:** MLP, Deep Learning
- **Méthodes ensemblistes:** Voting, Stacking, Bagging

In [ ]:
# 📦 Imports des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning
from sklearn.datasets import make_classification, load_wine, load_digits, load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.pipeline import Pipeline

# Algorithmes ML
from sklearn.ensemble import (
    RandomForestClassifier, 
    GradientBoostingClassifier, 
    AdaBoostClassifier,
    ExtraTreesClassifier,
    VotingClassifier
)
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

# Algorithmes avancés
try:
    import xgboost as xgb
    import lightgbm as lgb
    import catboost as cb
    ADVANCED_LIBS = True
    print("✅ Bibliothèques avancées disponibles: XGBoost, LightGBM, CatBoost")
except ImportError as e:
    ADVANCED_LIBS = False
    print(f"⚠️ Certaines bibliothèques manquantes: {e}")

# Configuration des graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

import warnings
warnings.filterwarnings('ignore')

print("🚀 Environnement configuré avec succès!")

## 📊 Génération et Exploration des Datasets

In [ ]:
# 📊 Création de datasets variés pour tester les algorithmes

def create_datasets():
    """Créer différents datasets pour tester la robustesse des modèles"""
    
    datasets = {}
    
    # 1. Dataset synthétique équilibré
    X_balanced, y_balanced = make_classification(
        n_samples=1000, n_features=20, n_informative=15, 
        n_redundant=5, n_classes=3, random_state=42
    )
    datasets['Synthétique Équilibré'] = (X_balanced, y_balanced)
    
    # 2. Dataset déséquilibré
    X_imbalanced, y_imbalanced = make_classification(
        n_samples=1000, n_features=15, n_informative=10,
        n_classes=3, weights=[0.7, 0.2, 0.1], random_state=42
    )
    datasets['Synthétique Déséquilibré'] = (X_imbalanced, y_imbalanced)
    
    # 3. Dataset haute dimension
    X_high_dim, y_high_dim = make_classification(
        n_samples=500, n_features=100, n_informative=50,
        n_redundant=20, n_classes=2, random_state=42
    )
    datasets['Haute Dimension'] = (X_high_dim, y_high_dim)
    
    # 4. Dataset réel - Wine
    wine = load_wine()
    datasets['Wine Quality'] = (wine.data, wine.target)
    
    # 5. Dataset réel - Breast Cancer
    cancer = load_breast_cancer()
    datasets['Breast Cancer'] = (cancer.data, cancer.target)
    
    return datasets

# Créer les datasets
datasets = create_datasets()

# Afficher les caractéristiques
print("📊 Caractéristiques des datasets:")
print("-" * 50)

for name, (X, y) in datasets.items():
    print(f"{name:25} | Samples: {X.shape[0]:4d} | Features: {X.shape[1]:3d} | Classes: {len(np.unique(y))}")
    print(f"{' ' * 25} | Distribution: {np.bincount(y)}")
    print()

## 🤖 Configuration des Algorithmes ML

In [ ]:
# 🤖 Configuration de tous les algorithmes ML

def get_ml_algorithms():
    """Retourner un dictionnaire de tous les algorithmes configurés"""
    
    algorithms = {
        # Algorithmes de base
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'Extra Trees': ExtraTreesClassifier(n_estimators=100, random_state=42),
        'Decision Tree': DecisionTreeClassifier(random_state=42),
        
        # Algorithmes linéaires
        'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
        'SGD Classifier': SGDClassifier(random_state=42, max_iter=1000),
        
        # SVM
        'SVM (RBF)': SVC(kernel='rbf', random_state=42),
        'SVM (Linear)': SVC(kernel='linear', random_state=42),
        
        # Algorithmes de proximité
        'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
        
        # Algorithmes probabilistes
        'Gaussian Naive Bayes': GaussianNB(),
        
        # Boosting classique
        'Gradient Boosting': GradientBoostingClassifier(random_state=42),
        'AdaBoost': AdaBoostClassifier(random_state=42),
        
        # Neural Networks
        'Neural Network (MLP)': MLPClassifier(
            hidden_layer_sizes=(100, 50), 
            random_state=42, 
            max_iter=500
        ),
    }
    
    # Ajouter les algorithmes avancés si disponibles
    if ADVANCED_LIBS:
        algorithms.update({
            'XGBoost': xgb.XGBClassifier(
                random_state=42, 
                verbosity=0,
                eval_metric='logloss'
            ),
            'LightGBM': lgb.LGBMClassifier(
                random_state=42, 
                verbose=-1,
                force_col_wise=True
            ),
            'CatBoost': cb.CatBoostClassifier(
                random_state=42, 
                verbose=False
            )
        })
    
    return algorithms

# Obtenir tous les algorithmes
algorithms = get_ml_algorithms()

print(f"🤖 {len(algorithms)} algorithmes configurés:")
for i, name in enumerate(algorithms.keys(), 1):
    print(f"{i:2d}. {name}")

## ⚡ Évaluation Complète des Modèles

In [ ]:
# ⚡ Fonction d'évaluation complète

def evaluate_algorithms(datasets, algorithms, cv_folds=5):
    """Évaluer tous les algorithmes sur tous les datasets"""
    
    results = []
    
    total_combinations = len(datasets) * len(algorithms)
    current = 0
    
    print(f"🚀 Début de l'évaluation: {total_combinations} combinaisons à tester")
    print("=" * 60)
    
    for dataset_name, (X, y) in datasets.items():
        print(f"\n📊 Dataset: {dataset_name}")
        
        # Standardisation des données
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Division train/test
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y, test_size=0.2, random_state=42, stratify=y
        )
        
        for algo_name, algorithm in algorithms.items():
            current += 1
            
            try:
                # Cross-validation
                cv_scores = cross_val_score(
                    algorithm, X_train, y_train, 
                    cv=StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42),
                    scoring='accuracy'
                )
                
                # Entraînement et test final
                algorithm.fit(X_train, y_train)
                test_score = algorithm.score(X_test, y_test)
                
                # Stocker les résultats
                results.append({
                    'Dataset': dataset_name,
                    'Algorithm': algo_name,
                    'CV_Mean': cv_scores.mean(),
                    'CV_Std': cv_scores.std(),
                    'Test_Accuracy': test_score,
                    'CV_Scores': cv_scores
                })
                
                print(f"  ✅ {algo_name:20s} | CV: {cv_scores.mean():.3f}±{cv_scores.std():.3f} | Test: {test_score:.3f}")
                
            except Exception as e:
                print(f"  ❌ {algo_name:20s} | Erreur: {str(e)[:50]}...")
                
            # Afficher le progrès
            if current % 10 == 0:
                print(f"\n🔄 Progrès: {current}/{total_combinations} ({100*current/total_combinations:.1f}%)")
    
    print("\n🎉 Évaluation terminée!")
    return pd.DataFrame(results)

# Lancer l'évaluation
results_df = evaluate_algorithms(datasets, algorithms)
print(f"\n📊 Résultats collectés: {len(results_df)} lignes")

## 📈 Analyse et Visualisation des Résultats

In [ ]:
# 📈 Analyse des résultats

# 1. Statistiques générales
print("📊 Statistiques Générales")
print("=" * 40)
print(f"Nombre total d'évaluations: {len(results_df)}")
print(f"Datasets testés: {results_df['Dataset'].nunique()}")
print(f"Algorithmes testés: {results_df['Algorithm'].nunique()}")
print(f"Accuracy moyenne: {results_df['Test_Accuracy'].mean():.3f}")
print(f"Écart-type: {results_df['Test_Accuracy'].std():.3f}")
print()

# 2. Top 5 des algorithmes
print("🏆 Top 5 des Algorithmes (par accuracy moyenne)")
print("-" * 50)
top_algorithms = results_df.groupby('Algorithm')['Test_Accuracy'].agg(['mean', 'std']).sort_values('mean', ascending=False)
for i, (algo, stats) in enumerate(top_algorithms.head().iterrows(), 1):
    print(f"{i}. {algo:25s} | {stats['mean']:.3f} ± {stats['std']:.3f}")
print()

# 3. Performance par dataset
print("📊 Performance par Dataset")
print("-" * 40)
dataset_performance = results_df.groupby('Dataset')['Test_Accuracy'].agg(['mean', 'std', 'max']).sort_values('mean', ascending=False)
for dataset, stats in dataset_performance.iterrows():
    print(f"{dataset:25s} | Moy: {stats['mean']:.3f} | Max: {stats['max']:.3f}")
print()

# Afficher le dataframe complet
print("📋 Tableau des résultats détaillés:")
display_df = results_df[['Dataset', 'Algorithm', 'CV_Mean', 'Test_Accuracy']].round(3)
display_df = display_df.sort_values(['Dataset', 'Test_Accuracy'], ascending=[True, False])
print(display_df.to_string(index=False))

In [ ]:
# 📊 Visualisations interactives avec Plotly

# 1. Heatmap des performances
pivot_data = results_df.pivot(index='Algorithm', columns='Dataset', values='Test_Accuracy')

fig1 = go.Figure(data=go.Heatmap(
    z=pivot_data.values,
    x=pivot_data.columns,
    y=pivot_data.index,
    colorscale='RdYlBu_r',
    text=np.round(pivot_data.values, 3),
    texttemplate="%{text}",
    textfont={"size":10},
    colorbar=dict(title="Accuracy")
))

fig1.update_layout(
    title="🔥 Heatmap des Performances - Accuracy par Algorithm/Dataset",
    xaxis_title="Datasets",
    yaxis_title="Algorithmes",
    height=600
)

fig1.show()

# 2. Box plot des performances par algorithme
fig2 = px.box(
    results_df, 
    x='Algorithm', 
    y='Test_Accuracy',
    title="📊 Distribution des Performances par Algorithme",
    color='Algorithm'
)
fig2.update_xaxis(tickangle=45)
fig2.update_layout(height=500, showlegend=False)
fig2.show()

# 3. Scatter plot CV vs Test
fig3 = px.scatter(
    results_df,
    x='CV_Mean',
    y='Test_Accuracy',
    color='Algorithm',
    size='CV_Std',
    hover_data=['Dataset'],
    title="🎯 Validation Croisée vs Performance Test"
)
fig3.add_shape(
    type="line",
    x0=results_df['CV_Mean'].min(),
    y0=results_df['CV_Mean'].min(),
    x1=results_df['CV_Mean'].max(),
    y1=results_df['CV_Mean'].max(),
    line=dict(color="red", dash="dash")
)
fig3.show()

In [ ]:
# 📊 Visualisations matplotlib pour rapport

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('🚀 Analyse Comparative des Algorithmes ML - Boubacar DABO', fontsize=16, fontweight='bold')

# 1. Performance moyenne par algorithme
algo_means = results_df.groupby('Algorithm')['Test_Accuracy'].mean().sort_values(ascending=True)
algo_means.plot(kind='barh', ax=axes[0,0], color='skyblue')
axes[0,0].set_title('📈 Performance Moyenne par Algorithme')
axes[0,0].set_xlabel('Accuracy Moyenne')

# 2. Distribution des accuracies
results_df['Test_Accuracy'].hist(bins=20, ax=axes[0,1], alpha=0.7, color='lightgreen')
axes[0,1].axvline(results_df['Test_Accuracy'].mean(), color='red', linestyle='--', label=f"Moyenne: {results_df['Test_Accuracy'].mean():.3f}")
axes[0,1].set_title('📊 Distribution des Performances')
axes[0,1].set_xlabel('Test Accuracy')
axes[0,1].legend()

# 3. Performance par dataset
dataset_means = results_df.groupby('Dataset')['Test_Accuracy'].mean().sort_values(ascending=True)
dataset_means.plot(kind='barh', ax=axes[1,0], color='orange')
axes[1,0].set_title('🎯 Difficulté des Datasets')
axes[1,0].set_xlabel('Accuracy Moyenne')

# 4. Variance des performances
algo_stds = results_df.groupby('Algorithm')['Test_Accuracy'].std().sort_values(ascending=True)
algo_stds.plot(kind='barh', ax=axes[1,1], color='pink')
axes[1,1].set_title('🎲 Stabilité des Algorithmes')
axes[1,1].set_xlabel('Écart-type')

plt.tight_layout()
plt.show()

## 🎯 Analyse Approfondie des Meilleurs Modèles

In [ ]:
# 🎯 Analyse détaillée des top 3 algorithmes

# Sélectionner les 3 meilleurs algorithmes
top_3_algos = results_df.groupby('Algorithm')['Test_Accuracy'].mean().nlargest(3).index.tolist()

print("🏆 Analyse des Top 3 Algorithmes")
print("=" * 40)

for i, algo in enumerate(top_3_algos, 1):
    algo_results = results_df[results_df['Algorithm'] == algo]
    
    print(f"\n{i}. {algo}")
    print("-" * 30)
    print(f"   Accuracy moyenne: {algo_results['Test_Accuracy'].mean():.4f}")
    print(f"   Écart-type: {algo_results['Test_Accuracy'].std():.4f}")
    print(f"   Min: {algo_results['Test_Accuracy'].min():.4f}")
    print(f"   Max: {algo_results['Test_Accuracy'].max():.4f}")
    
    # Meilleur dataset pour cet algorithme
    best_dataset = algo_results.loc[algo_results['Test_Accuracy'].idxmax(), 'Dataset']
    best_score = algo_results['Test_Accuracy'].max()
    print(f"   Meilleure performance: {best_score:.4f} sur {best_dataset}")
    
    # Pire dataset
    worst_dataset = algo_results.loc[algo_results['Test_Accuracy'].idxmin(), 'Dataset']
    worst_score = algo_results['Test_Accuracy'].min()
    print(f"   Pire performance: {worst_score:.4f} sur {worst_dataset}")

In [ ]:
# 📋 Création d'un rapport final

def generate_final_report(results_df):
    """Générer un rapport final complet"""
    
    report = {
        'total_evaluations': len(results_df),
        'datasets_count': results_df['Dataset'].nunique(),
        'algorithms_count': results_df['Algorithm'].nunique(),
        'overall_mean': results_df['Test_Accuracy'].mean(),
        'overall_std': results_df['Test_Accuracy'].std(),
        'best_algorithm': results_df.groupby('Algorithm')['Test_Accuracy'].mean().idxmax(),
        'best_algorithm_score': results_df.groupby('Algorithm')['Test_Accuracy'].mean().max(),
        'most_stable': results_df.groupby('Algorithm')['Test_Accuracy'].std().idxmin(),
        'easiest_dataset': results_df.groupby('Dataset')['Test_Accuracy'].mean().idxmax(),
        'hardest_dataset': results_df.groupby('Dataset')['Test_Accuracy'].mean().idxmin()
    }
    
    return report

# Générer le rapport
final_report = generate_final_report(results_df)

print("📋 RAPPORT FINAL - MACHINE LEARNING SHOWCASE")
print("=" * 50)
print(f"📊 Évaluations totales: {final_report['total_evaluations']}")
print(f"📁 Datasets testés: {final_report['datasets_count']}")
print(f"🤖 Algorithmes testés: {final_report['algorithms_count']}")
print(f"📈 Performance moyenne: {final_report['overall_mean']:.4f} ± {final_report['overall_std']:.4f}")
print()
print("🏆 RÉSULTATS CLÉS:")
print(f"   🥇 Meilleur algorithme: {final_report['best_algorithm']} ({final_report['best_algorithm_score']:.4f})")
print(f"   🎯 Plus stable: {final_report['most_stable']}")
print(f"   ✅ Dataset le plus facile: {final_report['easiest_dataset']}")
print(f"   💪 Dataset le plus difficile: {final_report['hardest_dataset']}")
print()
print("💡 RECOMMANDATIONS:")
print(f"   • Utiliser {final_report['best_algorithm']} pour de meilleures performances")
print(f"   • {final_report['most_stable']} offre la meilleure stabilité")
print("   • Tester plusieurs algorithmes pour chaque nouveau dataset")
print("   • Optimiser les hyperparamètres pour améliorer les performances")
print()
print("👨‍💼 Auteur: Boubacar DABO | ESIGELEC - Big Data & IA")
print("📧 Contact: dabom372@gmail.com")
print("🔗 Portfolio: https://bouba-dabo.github.io/portfolio")

---

## 🎓 Conclusions et Insights

### 📊 Résultats Obtenus

Cette analyse comparative a permis d'évaluer **15+ algorithmes de Machine Learning** sur **5 datasets différents**, représentant diverses complexités et caractéristiques:

- **Datasets synthétiques** pour contrôler la difficulté
- **Datasets réels** pour valider en conditions réelles
- **Différentes dimensions** et distributions de classes

### 🏆 Enseignements Clés

1. **Performance vs Stabilité**: Les algorithmes performants ne sont pas toujours les plus stables
2. **Importance du preprocessing**: La standardisation améliore significativement certains modèles
3. **No Free Lunch**: Aucun algorithme ne domine sur tous les datasets
4. **Gradient Boosting**: Généralement excellente performance sur datasets structurés

### 🚀 Perspectives d'Amélioration

- **Optimisation d'hyperparamètres** avec Grid/Random Search
- **Ensembles methods** pour combiner les meilleurs modèles
- **Feature engineering** adapté à chaque dataset
- **Validation plus robuste** avec des métriques multiples

### 💼 Applications Professionnelles

Cette approche systématique de comparaison d'algorithmes est essentielle en entreprise pour:
- **Sélection du bon modèle** pour chaque cas d'usage
- **Estimation de la performance** avant déploiement
- **Documentation** des choix techniques
- **Communication** avec les équipes métier

---

*Ce notebook démontre une maîtrise complète de l'écosystème Machine Learning Python et des bonnes pratiques d'évaluation comparative.*